# Análisis Financiero con Modelos de Lenguaje Natural (LLM)

## Introducción

Este notebook demuestra el uso de modelos de lenguaje natural (LLM) para el análisis financiero. Incluye la recopilación de datos, el preprocesamiento, el entrenamiento de modelos y la evaluación.

## Índice
1. [Importación de Librerías](#importacion-librerias)
2. [Carga de Datos](#carga-datos)
3. [Preprocesamiento de Datos](#preprocesamiento-datos)
4. [Entrenamiento del Modelo](#entrenamiento-modelo)
5. [Evaluación del Modelo](#evaluacion-modelo)
6. [Conclusiones](#conclusiones)

<a id="importacion-librerias"></a>

## 1. Importación de Librerías

En esta sección, importamos las librerías necesarias para el análisis.

In [1]:

from typing import Any

import os

from pydantic import BaseModel

from unstructured.partition.pdf import partition_pdf

In [2]:
import os
import json
import pprint
import openai
import chromadb

from chromadb.utils import embedding_functions
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

Definición del Nombre del Archivo PDF
Definimos el nombre del archivo PDF que será analizado. En este caso, estamos utilizando el informe "10-Q" de 2023 de Grainger.

Establecimiento del Directorio y Ruta de Salida
Obtenemos el directorio actual donde se está ejecutando el notebook y construimos la ruta completa hacia el archivo PDF de entrada. Esto asegura que podamos acceder al archivo sin importar dónde esté ubicado el notebook.

# Establecimiento del Directorio y Ruta de Salida
Obtenemos el directorio actual donde se está ejecutando el notebook y construimos la ruta completa hacia el archivo PDF de entrada. Esto asegura que podamos acceder al archivo sin importar dónde esté ubicado el notebook.

In [3]:
filename = "grainder report 10-q 2023.pdf"
# Directorio actual del notebook
current_directory = os.getcwd()

# Ruta completa del archivo de salida
output_path = os.path.join(current_directory, filename)

In [4]:
# Define parameters for Unstructured's library
strategy = "hi_res"  # Used for analyzing PDFs and extracting table structure
# Best model for table extraction. Other options are detectron2_onnx and chipper depending on file layout
model_name = "yolox"

In [6]:
pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 185.1 kB/s eta 0:00:00a 0:00:01
   ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/9.3 MB 4.9 kB/s eta 0:28:08m
ERROR: Exception:
Traceback (most recent call last):
  File "/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^

In [8]:
pip install tokenizers==0.13.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 kB/s eta 0:00:0000:0100:02
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
Note: you may need to restart the kernel to use updated packages.


In [9]:
import transformers
import tokenizers

print(transformers.__version__)
print(tokenizers.__version__)


4.32.1
0.13.3


In [10]:
elements = partition_pdf(filename=filename, strategy=strategy,
                         infer_table_structure=True, model_name=model_name)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Extracción de Elementos del PDF

En esta sección, utilizamos la biblioteca `unstructured` para particionar el contenido del PDF y extraer su estructura, incluyendo las tablas.

### Particionamiento del PDF

Utilizamos la función `partition_pdf` para analizar el PDF especificado y extraer sus elementos. Configuramos la estrategia de análisis como "hi_res" y usamos el modelo "yolox" para inferir la estructura de las tablas.

In [11]:
elements_to_json(elements, filename=f"{filename}.json")

In [13]:

#Aqui se procesan los datos, se extraen los elementos de la tabla y se escriben en un archivo de texto
# json.load(file) para cargar el contenido del archivo JSON en la variable data. Esto convierte el JSON en una estructura de datos de Python, como una lista o un diccionario, dependiendo del contenido del archivo JSON.
def process_json_file(input_filename):
    # Read the JSON file
    with open(input_filename, 'r') as file:
        data = json.load(file)

    # Iterate over the JSON data and extract required table elements
    extracted_elements = []
    # Si una entrada es de tipo "Table", extrae el texto HTML almacenado en entry["metadata"]["text_as_html"] y lo añade a la lista extracted_elements.
    for entry in data:
        if entry["type"] == "Table":
            extracted_elements.append(entry["metadata"]["text_as_html"])

    # Get the directory of the input filename
    output_dir = os.path.dirname(input_filename)

    # Construct the output filename
    output_filename = os.path.join(output_dir, "process-yolox.txt")

    # Write the extracted elements to the output file
    #Escritura de elementos extraídos en el archivo de salida:Abre el archivo de salida en modo de escritura ('w').Escribe cada elemento en extracted_elements en el archivo de salida, añadiendo dos saltos de línea después de cada elemento para separarlos visualmente.
    with open(output_filename, 'w') as output_file:
        for element in extracted_elements:
            # Adding two newlines for separation
            output_file.write(element + "\n\n")


## langchain section ##

In [8]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings

In [9]:
process_json_file(f"{filename}.json")

In [10]:
text_file = "process-yolox.txt"

In [11]:
loader = TextLoader(text_file)
documents = loader.load()

In [12]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

Created a chunk of size 2437, which is longer than the specified 1000
Created a chunk of size 1670, which is longer than the specified 1000
Created a chunk of size 3253, which is longer than the specified 1000
Created a chunk of size 2749, which is longer than the specified 1000
Created a chunk of size 3616, which is longer than the specified 1000
Created a chunk of size 3639, which is longer than the specified 1000
Created a chunk of size 1648, which is longer than the specified 1000
Created a chunk of size 1525, which is longer than the specified 1000
Created a chunk of size 1417, which is longer than the specified 1000
Created a chunk of size 1579, which is longer than the specified 1000
Created a chunk of size 1376, which is longer than the specified 1000
Created a chunk of size 1585, which is longer than the specified 1000
Created a chunk of size 1595, which is longer than the specified 1000


In [13]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings()

/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
db = Chroma.from_documents(docs, embeddings)

In [23]:
# Initialize your model and retriever
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [21]:

# List of questions
questions2 = [
    {
        "section": "Resumen Ejecutivo",
        "query": "Proporciona una visión general de la empresa, destacando los aspectos más importantes de su desempeño financiero en el último año."
    },
    {
        "section": "Análisis de Ingresos",
        "query": "Analiza los ingresos operativos de la empresa y cómo estos han variado a lo largo del tiempo. Es importante destacar las principales fuentes de ingresos y cualquier cambio significativo en las operaciones."
    },
    {
        "section": "Análisis de Costos y Gastos",
        "query": "Desglosa los costos operativos y gastos de la empresa. Incluye detalles sobre los costos de bienes vendidos, gastos administrativos y de ventas, y cualquier otro gasto relevante."
    },
    {
        "section": "Análisis de Rentabilidad",
        "query": "Examina los ingresos netos para entender la rentabilidad de la empresa. Incluye cualquier factor que haya impactado significativamente en los ingresos netos, como costos extraordinarios o cambios en la estructura de costos."
    },
    {
        "section": "Análisis de Balance General",
        "query": "Analiza los activos y pasivos de la empresa, destacando cualquier cambio significativo en los últimos trimestres. Incluye detalles sobre activos corrientes, pasivos corrientes y el capital de trabajo neto."
    },
    {
        "section": "Análisis de Flujos de Efectivo",
        "query": "Evalúa los flujos de efectivo de la empresa, incluyendo las actividades operativas, de inversión y de financiación. Destaca cualquier cambio significativo en los flujos de efectivo."
    },
    {
        "section": "Factores de Riesgo",
        "query": "Identifica y analiza los principales factores de riesgo mencionados en el reporte 10-K. Incluye riesgos operativos, financieros y de mercado."
    },
    {
        "section": "Perspectivas Futuras",
        "query": "Resume las perspectivas futuras y la estrategia de la empresa según lo descrito en el informe de gestión del 10-K."
    }
]

# Store responses in output_list
output_list = []

for question in questions2:
    response = qa_chain({"query": question["query"]})
    output_list.append({
        "section": question["section"],
        "query": question["query"],
        "response": response["result"]
    })

# Print the output_list to verify the structure
print(output_list)

[{'section': 'Resumen Ejecutivo', 'query': 'Proporciona una visión general de la empresa, destacando los aspectos más importantes de su desempeño financiero en el último año.', 'response': 'Lo siento, pero la información proporcionada no incluye detalles específicos sobre el desempeño financiero de la empresa en el último año, como los ingresos totales, las ganancias netas, los gastos operativos, etc. Solo se proporcionan los títulos de los estados financieros y algunas cifras relacionadas con las actividades de efectivo, pero no se proporciona suficiente contexto o detalles para proporcionar una visión general completa del desempeño financiero de la empresa.'}, {'section': 'Análisis de Ingresos', 'query': 'Analiza los ingresos operativos de la empresa y cómo estos han variado a lo largo del tiempo. Es importante destacar las principales fuentes de ingresos y cualquier cambio significativo en las operaciones.', 'response': 'Lo siento, pero los datos proporcionados no incluyen suficient

In [16]:
# List of questions for numerical data
questions3 = [
    "¿Cuáles fueron los ingresos operativos de la empresa en 2022?",
    "¿Cuáles fueron los ingresos operativos de la empresa en 2023?",
    "¿Cuáles fueron los gastos de venta, generales y administrativos en 2022?",
    "¿Cuáles fueron los gastos de venta, generales y administrativos en 2023?",
    "¿Cuál fue el ingreso neto en 2022?",
    "¿Cuál fue el ingreso neto en 2023?",
    "¿Cuál fue el flujo de caja operativo en 2022?",
    "¿Cuál fue el flujo de caja operativo en 2023?"
]

# Store responses in a list
responses2 = []

for query in questions3:
    response = qa_chain({"query": query})
    responses2.append(response["result"])

# Print the responses to verify
for q, r in zip(questions3, responses2):
    print(f"Pregunta: {q}\nRespuesta: {r}\n")

/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Pregunta: ¿Cuáles fueron los ingresos operativos de la empresa en 2022?
Respuesta: Lo siento, pero la información proporcionada no incluye los ingresos operativos de la empresa para el año 2022.

Pregunta: ¿Cuáles fueron los ingresos operativos de la empresa en 2023?
Respuesta: Los ingresos operativos de la empresa en 2023 fueron de $2,008.

Pregunta: ¿Cuáles fueron los gastos de venta, generales y administrativos en 2022?
Respuesta: Los gastos de venta, generales y administrativos en 2022 fueron:

- $152 en la primera tabla
- $2,160 en la segunda tabla
- $741 en la tercera tabla
- $441 en la cuarta tabla

Pregunta: ¿Cuáles fueron los gastos de venta, generales y administrativos en 2023?
Respuesta: Los gastos de venta, generales y administrativos en 2023 fueron:

- $161 en la primera tabla
- $2,380 en la segunda tabla
- $806 en la tercera tabla
- $476 en la cuarta tabla

Pregunta: ¿Cuál fue el ingreso neto en 2022?
Respuesta: Lo siento, pero la información proporcionada no incluye deta

Resumen Ejecutivo:
Proporciona una visión general de la empresa, destacando los aspectos más importantes de su desempeño financiero.

Análisis de Ingresos:
Discute el rendimiento de los ingresos de la empresa, tanto para el trimestre específico (Q4 FY23) como para el año completo (FY23). Compara los ingresos con periodos anteriores para identificar tendencias de crecimiento o declive.


Ingresos Operativos:
Analiza los ingresos operativos de la empresa y cómo estos han variado a lo largo del tiempo. Es importante destacar las principales fuentes de ingresos y cualquier cambio significativo en las operaciones.


Ingresos Netos:
Examina los ingresos netos para entender la rentabilidad de la empresa. Incluye cualquier factor que haya impactado significativamente en los ingresos netos, como costos extraordinarios o cambios en la estructura de costos.

Indicadores Financieros Claves:
Incluye y analiza otros indicadores financieros importantes, como el ratio actual, para proporcionar una imagen más completa de la salud financiera de la empresa.

Conclusiones y Recomendaciones:
Resume tus hallazgos y proporciona recomendaciones basadas en el análisis. Esto puede incluir sugerencias sobre cómo la empresa puede mejorar su rendimiento financiero o áreas donde podrían existir riesgos potenciales.